In [ ]:
# Builtin modules
from itertools import chain
import random
import warnings
import sys
import os

# Images
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from PIL import ImageFile
import matplotlib.pyplot as plt
import numpy as np
import cv2

# Deep learning
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Dropout, Lambda
from keras.layers.merge import concatenate
from keras.layers import Input
from keras.models import Model, load_model
from keras import backend as K
from tensorflow.python.util import deprecation
import tensorflow as tf

# Ignore warnings
warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
ImageFile.LOAD_TRUNCATED_IMAGES = True
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
deprecation._PRINT_DEPRECATION_WARNINGS = False

In [2]:
# Set the parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
TRAIN_PATH = 'data/train/'
TEST_PATH = 'data/test/'

random.seed = np.random.seed = 42

In [3]:
# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

# ---------------------------------------------------

# Get and resize train images and masks
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

for n, id_ in enumerate(train_ids):
    path = TRAIN_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask
    
# ---------------------------------------------------
    
# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []

for n, id_ in enumerate(test_ids):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:, :, :IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img
    

print("X_train", X_train.shape)
print("Y_train", Y_train.shape)

X_train (33, 128, 128, 3)
Y_train (33, 128, 128, 1)


In [4]:
# Define IoU function 
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [5]:
# Build U-Net model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 128, 128, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 16) 448         lambda_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 128, 128, 16) 0           conv2d_1[0][0]                   
___________________________________________________________________________________________

In [6]:
# Fit model
earlystopper = EarlyStopping(patience=50, verbose=1)
checkpointer = ModelCheckpoint('test/model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
results = model.fit(X_train, 
                    Y_train,
                    validation_split=0.1,
                    batch_size=16,
                    epochs=50,
                    callbacks=[earlystopper, checkpointer])

#Predict on train, val and test
model = load_model('test/model-dsbowl2018-1.h5', custom_objects={'mean_iou': mean_iou})
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                      (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))


Train on 29 samples, validate on 4 samples
Epoch 1/50
29/29 [==============================] - 5s 172ms/step - loss: 0.7494 - mean_iou: 0.1936 - val_loss: 0.6806 - val_mean_iou: 0.3940

Epoch 00001: val_loss improved from inf to 0.68064, saving model to test/model-dsbowl2018-1.h5
Epoch 2/50
29/29 [==============================] - 3s 87ms/step - loss: 0.5746 - mean_iou: 0.3979 - val_loss: 0.6962 - val_mean_iou: 0.4005

Epoch 00002: val_loss did not improve from 0.68064
Epoch 3/50
29/29 [==============================] - 2s 86ms/step - loss: 0.5051 - mean_iou: 0.3971 - val_loss: 0.7139 - val_mean_iou: 0.4026

Epoch 00003: val_loss did not improve from 0.68064
Epoch 4/50
29/29 [==============================] - 3s 107ms/step - loss: 0.4717 - mean_iou: 0.4014 - val_loss: 0.7467 - val_mean_iou: 0.4117

Epoch 00004: val_loss did not improve from 0.68064
Epoch 5/50
29/29 [==============================] - 3s 93ms/step - loss: 0.4637 - mean_iou: 0.4092 - val_loss: 0.6533 - val_mean_iou: 0.41


Epoch 00038: val_loss improved from 0.21540 to 0.20994, saving model to test/model-dsbowl2018-1.h5
Epoch 39/50
29/29 [==============================] - 3s 93ms/step - loss: 0.1110 - mean_iou: 0.6508 - val_loss: 0.2336 - val_mean_iou: 0.6533

Epoch 00039: val_loss did not improve from 0.20994
Epoch 40/50
29/29 [==============================] - 2s 85ms/step - loss: 0.1120 - mean_iou: 0.6548 - val_loss: 0.2420 - val_mean_iou: 0.6577

Epoch 00040: val_loss did not improve from 0.20994
Epoch 41/50
29/29 [==============================] - 2s 84ms/step - loss: 0.1063 - mean_iou: 0.6593 - val_loss: 0.2567 - val_mean_iou: 0.6617

Epoch 00041: val_loss did not improve from 0.20994
Epoch 42/50
29/29 [==============================] - 2s 83ms/step - loss: 0.1064 - mean_iou: 0.6630 - val_loss: 0.2799 - val_mean_iou: 0.6653

Epoch 00042: val_loss did not improve from 0.20994
Epoch 43/50
29/29 [==============================] - 2s 83ms/step - loss: 0.1013 - mean_iou: 0.6668 - val_loss: 0.2899 - val